## Import Modul

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import os
from PIL import  Image
%matplotlib inline
import itertools
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

In [ ]:
dataset = pd.read_csv("C:/Users/Kuncoro/Documents/python/weatherAUS.csv")

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.apply(lambda column: sum(column.isnull()))

## Remove Null

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.apply(lambda column: sum(column.isnull()))

In [ ]:
dataset.info()

In [ ]:
dataset.head()

## Change Datetime

In [ ]:
dataset['Date'] = pd.to_datetime(dataset['Date'])

In [ ]:
dataset['Year'] = dataset['Date'].dt.year

In [ ]:
dataset['Month'] = dataset['Date'].dt.month

In [ ]:
dataset['Day'] = dataset['Date'].dt.day

In [ ]:
dataset.head()

## EDA

In [ ]:
#change object to numeric dtype
dataset["Date"] = pd.to_numeric(dataset["Date"])

#Separating rain and non rain customers
rain     = dataset[dataset["RainTomorrow"] == "Yes"]
not_rain = dataset[dataset["RainTomorrow"] == "No"]

#Separating columns
Id_col     = ['Date']
target_col = ["RainTomorrow"]
cat_cols   = dataset.nunique()[dataset.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in dataset.columns if x not in cat_cols + target_col + Id_col]

In [ ]:
dataset.info()

In [ ]:
#labels
lab = dataset["RainTomorrow"].value_counts().keys().tolist()
#values
val = dataset["RainTomorrow"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Rain Tomorrow data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [ ]:
#function  for pie plot for customer attrition types
def plot_pie(column) :
    
    trace1 = go.Pie(values  = rain[column].value_counts().values.tolist(),
                    labels  = rain[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "Rain Tomorrow",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = not_rain[column].value_counts().values.tolist(),
                    labels  = not_rain[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "Non Rain Tomorrow" 
                   )


    layout = go.Layout(dict(title = column + " distribution in Rain Tomorrow  ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "Rain Tomorrow",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "Non Rain Tomorrow",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    py.iplot(fig)


#function  for histogram for customer attrition types
def histogram(column) :
    trace1 = go.Histogram(x  = rain[column],
                          histnorm= "percent",
                          name = "Rain Tomorrow",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = not_rain[column],
                          histnorm = "percent",
                          name = "Non Rain Tomorrow",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data = [trace1,trace2]
    layout = go.Layout(dict(title =column + " distribution in Rain Tomorrow ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data,layout=layout)
    
    py.iplot(fig)
    

#for all categorical columns plot pie
for i in cat_cols :
    plot_pie(i)

#for all categorical columns plot histogram    
for i in num_cols :
    histogram(i)


In [ ]:
dataset.nunique()

## PREDICTION RAIN TOMORROW

In [ ]:
dataset = dataset[['MinTemp', 'MaxTemp', 'Evaporation',
        'Sunshine', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow']].reset_index(drop=True)

## mempengaruhi data
## Risk_MM

## object 
## Date, Location, WindGustDir(object) , WindDir9am(object) , WindDir3pm(object), Year, Month, Day 

In [ ]:
dataset.head()

## Membuat colom baru 

In [ ]:
dataset['RainToday'] = 'Today_' + dataset['RainToday'].astype(str)  

today = pd.get_dummies(dataset['RainToday'])  
weather = dataset.join(today)


In [ ]:
weather = weather.drop(['RainToday'], axis=1)

In [ ]:
weather.head()

## Merubah Column RainTomorrow ke Number 

In [ ]:
weather.RainTomorrow.replace(('Yes', 'No'), (1, 0), inplace=True)

In [ ]:
weather.head()

In [ ]:
weather_of_AUS = len(weather)
number_of_rain = len(weather[weather['RainTomorrow'] == 1])
number_of_no_rain = len(weather[weather['RainTomorrow'] == 0])
print('Number of Weater in AUS', weather_of_AUS)
print('Number of rain :%d (%.2f %%)' % (number_of_rain,number_of_rain/weather_of_AUS*100))
print('Number of no rain :%d (%.2f %%)' % (number_of_no_rain,number_of_no_rain/weather_of_AUS*100))

## Correlation Matrix

In [ ]:
#correlation
correlation = weather.corr()
#tick labels
matrix_cols = correlation.columns.tolist()
#convert to array
corr_array  = np.array(correlation)

#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Split Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = weather.drop(['RainTomorrow'], axis=1)

In [ ]:
target = weather['RainTomorrow']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.2)

In [ ]:
print(len(weather))
print('Number x_train :%d (%.2f %%)' % (len(x_train),len(x_train)/len(weather)*100))
print('Number x_test :%d (%.2f %%)' % (len(x_test),len(x_test)/len(weather)*100))

In [ ]:
# train, test, validate = np.split(weather.sample(frac=1), [int(.6*len(weather)), int(.8*len(weather))])
# print(train.shape, test.shape, validate.shape)

In [ ]:
# y_train = train['RainTomorrow']
# x_train = train.drop(['RainTomorrow'], axis=1)
# y_test = test['RainTomorrow']
# x_test = test.drop(['RainTomorrow'], axis=1)
# y_validate = validate['RainTomorrow']
# x_validate = validate.drop(['RainTomorrow'], axis=1)

In [ ]:
y_test.mean()

In [ ]:
y_train.mean()

## Feature Importances

In [ ]:
rf = RandomForestClassifier() #mencacri corelasi yang paling bagus dr left
rf.fit(x_train, y_train)
print('Feature sorted by their score: ')
print(sorted(zip(map(lambda x : round(x,4), rf.feature_importances_),x_train),reverse=True))

In [ ]:
all_vars = x_train.columns.tolist()
top_2_vars = ['Humidity3pm','Sunshine']
bottom_vars = [cols for cols in all_vars if cols not in top_2_vars]

x_train = x_train.drop(bottom_vars,axis=1)
x_test = x_test.drop(bottom_vars,axis=1)

## LOGISTIC Regresion

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
from yellowbrick.classifier import DiscriminationThreshold
#splitting train and test data 
train,test = train_test_split(weather,test_size = .25 ,random_state = 111)
    
##seperating dependent and independent variables
cols    = [i for i in weather.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

#Function attributes
#dataframe     - processed dataframe
#Algorithm     - Algorithm used 
#training_x    - predictor variables dataframe(training)
#testing_x     - predictor variables dataframe(testing)
#training_y    - target variable(training)
#training_y    - target variable(testing)
#cf - ["coefficients","features"](cooefficients for logistic 
                                 #regression,features for tree based models)

#threshold_plot - if True returns threshold plot for model
    
def telecom_churn_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y,cols,cf,threshold_plot) :
    
    #model
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    #coeffs
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    print (algorithm)
    print ("\n Classification report : \n",classification_report(testing_y,predictions))
    print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    #plot coeffs
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    
    if threshold_plot == True : 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()
        
logit  = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

telecom_churn_prediction(logit,train_X,test_X,train_Y,test_Y,
                         cols,"coefficients",threshold_plot = True)



In [ ]:
logit_model = LogisticRegression()
logit_model = logit_model.fit(x_train, y_train)
predicted = pd.DataFrame(logit_model.predict(x_test))
metrics.accuracy_score(y_test, predicted) ## HOLD OUT

In [ ]:
rf_cv_scores = cross_validate(LogisticRegression(), x_test, y_test, cv=10, scoring='accuracy') #CROSS VALIDATE
np.mean(rf_cv_scores['test_score'])

## RANDOM FORREST

In [ ]:
randomF = RandomForestClassifier()
randomF_model = randomF.fit(x_train, y_train)
randomF_model.score(x_train,y_train) 
predicted = pd.DataFrame(randomF_model.predict(x_test))
metrics.accuracy_score(y_test, predicted) ## HOLD OUT

In [ ]:
randomF_cv_scores = cross_validate(RandomForestClassifier(), x_test, y_test, cv=10, scoring='accuracy') #CROSS VALIDATE
np.mean(randomF_cv_scores['test_score'])

## k-NN

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)
knn_model.score(x_train,y_train) 
predicted = pd.DataFrame(knn_model.predict(x_test))
metrics.accuracy_score(y_test, predicted ) ## HOLD OUT

In [ ]:
knn_cv_scores = cross_validate(KNeighborsClassifier(), x_test, y_test, cv=10, scoring='accuracy') #CROSS VALIDATE
np.mean(knn_cv_scores['test_score'])

## SVM

In [ ]:
# svm_model = SVC(probability=True)  
# svm_model = svm_model.fit(x_train, y_train)  
# svm_model.score(x_train, y_train)
# predicted = pd.DataFrame(svm_model.predict(x_test))
# metrics.accuracy_score(y_test, predicted ) ## HOLD OUT

In [ ]:
# rf_cv_scores = cross_validate(SVC(probability=True), x_test, y_test, cv=10, scoring='recall')  
# np.mean(rf_cv_scores['test_score'])

## NEW DATA RAIN TOMORROW

In [ ]:
x_train.head(1)

In [ ]:
new_data = pd.DataFrame.from_dict({
    'Sunshine':[0.3],
    'Humidity3pm':[43.0]
})
logit_model.predict(new_data)